# 🏃‍♂️ Criador de Plano de Treino Personalizado

## Bem-vindo ao seu assistente de treino inteligente!

Este notebook irá coletar suas informações pessoais para criar um plano de treino completamente personalizado, adaptado às suas necessidades, objetivos e limitações.

**Preencha todas as seções abaixo cuidadosamente.**

---

In [ ]:
# Importar módulos necessários
from user_profile import UserProfile, RaceGoal
from training_zones import TrainingZones, RaceTime
from plan_generator import PlanGenerator
from datetime import datetime, date, timedelta

print("✓ Módulos carregados com sucesso!")
print("\nPreencha as células abaixo para criar seu perfil personalizado.")

---
## 1️⃣ Informações Pessoais

Vamos começar com suas informações básicas.

In [ ]:
# ===== PREENCHA AQUI =====

# Seu nome
NOME = "Seu Nome"

# Sua idade (em anos)
IDADE = 30

# Seu peso (em kg)
PESO_KG = 70.0

# Sua altura (em cm)
ALTURA_CM = 175.0

# Sexo: "M" (masculino), "F" (feminino) ou "" (não especificar)
SEXO = "M"

# ===== FIM DO PREENCHIMENTO =====

# Calcular IMC
profile = UserProfile(
    name=NOME,
    age=IDADE,
    weight_kg=PESO_KG,
    height_cm=ALTURA_CM,
    gender=SEXO
)

print(f"✓ Nome: {NOME}")
print(f"✓ Idade: {IDADE} anos")
print(f"✓ Peso: {PESO_KG}kg | Altura: {ALTURA_CM}cm")
print(f"✓ IMC: {profile.calculate_bmi()} - {profile.get_bmi_category()}")

---
## 2️⃣ Experiência em Corrida

Conte-nos sobre sua experiência com corrida.

In [ ]:
# ===== PREENCHA AQUI =====

# Há quantos anos você corre? (pode usar decimais, ex: 2.5)
ANOS_CORRENDO = 2.0

# Qual sua kilometragem semanal ATUAL? (em km)
KM_SEMANAL_ATUAL = 30.0

# Qual seu nível de experiência?
# Opções: "beginner" (iniciante), "intermediate" (intermediário), "advanced" (avançado)
NIVEL_EXPERIENCIA = "intermediate"

# ===== FIM DO PREENCHIMENTO =====

profile.years_running = ANOS_CORRENDO
profile.current_weekly_km = KM_SEMANAL_ATUAL
profile.experience_level = NIVEL_EXPERIENCIA

print(f"✓ Experiência: {ANOS_CORRENDO} anos correndo")
print(f"✓ Nível: {NIVEL_EXPERIENCIA}")
print(f"✓ Kilometragem semanal atual: {KM_SEMANAL_ATUAL}km")

---
## 3️⃣ Objetivos e Provas

Defina suas metas de corrida.

In [ ]:
# ===== PREENCHA AQUI =====

# PROVA PRINCIPAL
PROVA_DISTANCIA = "10K"  # Opções: "5K", "10K", "Half Marathon", "Marathon"
PROVA_DATA = "2025-04-15"  # Formato: "AAAA-MM-DD"
PROVA_NOME = "Corrida da Cidade"  # Nome da prova (opcional)
PROVA_LOCAL = "São Paulo"  # Local da prova (opcional)
PROVA_TEMPO_META = "45:00"  # Tempo meta em MM:SS ou HH:MM:SS (opcional, deixe "" se não tiver)

# OBJETIVOS SECUNDÁRIOS
# Selecione os que se aplicam (mantenha os que quiser, delete os que não quiser)
OBJETIVOS_SECUNDARIOS = [
    "Performance/Tempo",
    "Saúde Geral",
    # "Perda de Peso",
    # "Manutenção de Peso",
    # "Bem-estar Mental",
    # "Socialização",
    "Desafio Pessoal",
    # "Qualificação para Prova"
]

# ===== FIM DO PREENCHIMENTO =====

# Criar prova principal
prova_data = datetime.strptime(PROVA_DATA, "%Y-%m-%d").date()
profile.main_race = RaceGoal(
    distance=PROVA_DISTANCIA,
    date=prova_data,
    name=PROVA_NOME,
    location=PROVA_LOCAL,
    is_main_goal=True,
    target_time=PROVA_TEMPO_META if PROVA_TEMPO_META else None
)

profile.secondary_objectives = OBJETIVOS_SECUNDARIOS

print("✓ Prova Principal:")
print(f"  {PROVA_DISTANCIA} - {PROVA_NOME}")
print(f"  Data: {prova_data.strftime('%d/%m/%Y')}")
if PROVA_TEMPO_META:
    print(f"  Meta: {PROVA_TEMPO_META}")
print(f"\n✓ Objetivos Secundários: {', '.join(OBJETIVOS_SECUNDARIOS)}")

# Calcular quantas semanas até a prova
hoje = date.today()
semanas_ate_prova = (prova_data - hoje).days // 7
print(f"\n⏰ Tempo até a prova: {semanas_ate_prova} semanas")

### 3.1 Provas Teste (Opcional)

Se você planeja fazer provas teste antes da prova principal, adicione-as aqui.

In [ ]:
# ===== PREENCHA AQUI =====

# Lista de provas teste (adicione quantas quiser)
# Formato: (distância, data, nome)
PROVAS_TESTE = [
    # ("5K", "2025-03-01", "Prova Teste 5K"),
    # ("10K", "2025-03-22", "Prova Preparatória"),
]

# ===== FIM DO PREENCHIMENTO =====

for dist, data, nome in PROVAS_TESTE:
    test_date = datetime.strptime(data, "%Y-%m-%d").date()
    profile.test_races.append(RaceGoal(
        distance=dist,
        date=test_date,
        name=nome,
        is_main_goal=False
    ))

if profile.test_races:
    print("✓ Provas Teste:")
    for race in profile.test_races:
        print(f"  • {race.distance} em {race.date.strftime('%d/%m/%Y')} - {race.name}")
else:
    print("✓ Sem provas teste configuradas")

---
## 4️⃣ Disponibilidade de Tempo

Quanto tempo você tem disponível para treinar?

In [ ]:
# ===== PREENCHA AQUI =====

# Quantos dias por semana você pode treinar?
DIAS_POR_SEMANA = 4

# Quantas horas por dia você tem disponível para treinar?
HORAS_POR_DIA = 1.0

# Qual seu horário preferido?
# Opções: "morning" (manhã), "afternoon" (tarde), "evening" (noite)
HORARIO_PREFERIDO = "morning"

# Onde você prefere treinar? (selecione os que se aplicam)
LOCAIS_PREFERIDOS = [
    "road",  # rua
    # "track",  # pista
    # "trail",  # trilha
    # "treadmill",  # esteira
]

# ===== FIM DO PREENCHIMENTO =====

profile.days_per_week = DIAS_POR_SEMANA
profile.hours_per_day = HORAS_POR_DIA
profile.preferred_time = HORARIO_PREFERIDO
profile.preferred_location = LOCAIS_PREFERIDOS

tempo_total = profile.get_weekly_time_budget()

print(f"✓ Disponibilidade: {DIAS_POR_SEMANA} dias/semana")
print(f"✓ Tempo por dia: {HORAS_POR_DIA}h")
print(f"✓ Tempo total semanal: {tempo_total}h")
print(f"✓ Horário preferido: {HORARIO_PREFERIDO}")
print(f"✓ Locais preferidos: {', '.join(LOCAIS_PREFERIDOS)}")

# Alerta se tempo for muito limitado
if tempo_total < 3:
    print("\n⚠️  ATENÇÃO: Você tem menos de 3 horas semanais disponíveis.")
    print("   O plano será adaptado com treinos mais curtos e eficientes.")

---
## 5️⃣ Zonas de Treinamento

Para calcular suas zonas de treino personalizadas, precisamos de seus tempos recentes em provas.

In [ ]:
# ===== PREENCHA AQUI =====

# Seus tempos recentes em provas (formato MM:SS ou HH:MM:SS)
# Adicione apenas os que você TEM. Deixe como "" os que não tem.
TEMPO_5K = "22:30"  # Ex: "22:30" para 22min30s
TEMPO_10K = "47:15"  # Ex: "47:15" para 47min15s
TEMPO_21K = ""  # Ex: "1:45:30" para 1h45min30s
TEMPO_42K = ""  # Ex: "3:45:00" para 3h45min

# Método de cálculo das zonas
# Opções: "jack_daniels" (mais preciso, baseado em VO2max) ou "critical_velocity"
METODO_ZONAS = "jack_daniels"

# ===== FIM DO PREENCHIMENTO =====

# Adicionar tempos ao perfil
if TEMPO_5K:
    profile.recent_race_times["5K"] = TEMPO_5K
if TEMPO_10K:
    profile.recent_race_times["10K"] = TEMPO_10K
if TEMPO_21K:
    profile.recent_race_times["21K"] = TEMPO_21K
if TEMPO_42K:
    profile.recent_race_times["42K"] = TEMPO_42K

profile.zones_calculation_method = METODO_ZONAS

print("✓ Tempos Recentes:")
for dist, tempo in profile.recent_race_times.items():
    print(f"  {dist}: {tempo}")
print(f"\n✓ Método de cálculo: {METODO_ZONAS}")

if not profile.recent_race_times:
    print("\n⚠️  ATENÇÃO: Nenhum tempo de prova informado.")
    print("   O plano será criado sem zonas personalizadas de pace.")

### 5.1 Frequência Cardíaca (Opcional)

Se você conhece sua FC de repouso e máxima, pode informá-las aqui.

In [ ]:
# ===== PREENCHA AQUI =====

# FC de repouso (bpm) - medida ao acordar, antes de levantar
FC_REPOUSO = None  # Ex: 55, ou deixe None se não souber

# FC máxima (bpm) - use None para estimar automaticamente
FC_MAXIMA = None  # Ex: 185, ou deixe None para estimar

# ===== FIM DO PREENCHIMENTO =====

profile.hr_resting = FC_REPOUSO
profile.hr_max = FC_MAXIMA

fc_max_estimada = profile.estimate_hr_max()

print("✓ Frequência Cardíaca:")
if FC_REPOUSO:
    print(f"  Repouso: {FC_REPOUSO} bpm")
if fc_max_estimada:
    if FC_MAXIMA:
        print(f"  Máxima: {fc_max_estimada} bpm")
    else:
        print(f"  Máxima (estimada): {fc_max_estimada} bpm")

if not FC_REPOUSO and not FC_MAXIMA:
    print("  Não configurada (opcional)")

---
## 6️⃣ Histórico de Lesões

Importante para adaptar seu plano e prevenir recidivas.

In [ ]:
# ===== PREENCHA AQUI =====

# Lesões ATUAIS (que você tem AGORA)
LESOES_ATUAIS = [
    # "Fascite Plantar",
    # "Canelite (Periostite Tibial)",
]

# Lesões PRÉVIAS (que você já teve no passado)
LESOES_PREVIAS = [
    # "Canelite (Periostite Tibial)",
    # "Tendinite de Aquiles",
]

# ===== FIM DO PREENCHIMENTO =====

# Opções disponíveis (para referência):
print("Opções de lesões comuns:")
for i, lesao in enumerate(UserProfile.COMMON_INJURIES, 1):
    print(f"  {i}. {lesao}")

profile.current_injuries = LESOES_ATUAIS
profile.previous_injuries = LESOES_PREVIAS

print("\n✓ Histórico de Lesões:")
if LESOES_ATUAIS:
    print(f"  Lesões Atuais: {', '.join(LESOES_ATUAIS)}")
else:
    print("  Lesões Atuais: Nenhuma")

if LESOES_PREVIAS:
    print(f"  Lesões Prévias: {', '.join(LESOES_PREVIAS)}")
else:
    print("  Lesões Prévias: Nenhuma")

# Avaliar risco
risco = profile.get_injury_risk_level()
print(f"\n✓ Nível de Risco de Lesão: {risco}")

if risco == "Alto":
    print("  ⚠️  Seu plano será mais conservador para prevenir lesões.")

---
## 7️⃣ Equipamentos Disponíveis

Quais equipamentos você tem acesso?

In [ ]:
# ===== PREENCHA AQUI =====

# Selecione os equipamentos que você TEM acesso
EQUIPAMENTOS = [
    "Relógio GPS/Smartwatch",
    # "Monitor de Frequência Cardíaca",
    # "Acesso a Pista de Atletismo",
    # "Esteira",
    # "Rolo de Massagem/Foam Roller",
    # "Faixas de Resistência",
    # "Academia",
]

# ===== FIM DO PREENCHIMENTO =====

profile.available_equipment = EQUIPAMENTOS

print("✓ Equipamentos Disponíveis:")
if EQUIPAMENTOS:
    for eq in EQUIPAMENTOS:
        print(f"  • {eq}")
else:
    print("  Nenhum equipamento especial")

---
## 8️⃣ Resumo do Perfil

Veja um resumo completo de todas as informações coletadas.

In [ ]:
# Atualizar data de última modificação
profile.last_updated = datetime.now()

# Mostrar perfil completo
print(profile)

# Verificar se precisa de modificações
needs_mod, modifications = profile.needs_modified_plan()
if needs_mod:
    print("\n" + "="*70)
    print("⚠️  SEU PLANO SERÁ ADAPTADO COM BASE NOS SEGUINTES FATORES:")
    print("="*70)
    for mod in modifications:
        print(f"  • {mod}")

---
## 9️⃣ Salvar Perfil

Salve seu perfil para uso futuro.

In [ ]:
# Salvar perfil
filename = "meu_perfil.json"
profile.save_to_file(filename)

print(f"✓ Perfil salvo em: {filename}")
print(f"✓ Você pode carregar este perfil mais tarde usando:")
print(f"  profile = UserProfile.load_from_file('{filename}')")

---
## 🔟 Gerar Plano de Treino Personalizado

Agora vamos criar seu plano baseado em todo o perfil!

In [ ]:
# Gerar plano usando o perfil completo do usuário
print(f"\n{'='*70}")
print("GERANDO SEU PLANO PERSONALIZADO")
print(f"{'='*70}")

# Calcular duração do plano
hoje = date.today()
dias_ate_prova = (profile.main_race.date - hoje).days
semanas_disponiveis = dias_ate_prova // 7

print(f"Semanas até a prova: {semanas_disponiveis}")
print(f"Nível: {profile.experience_level}")
print(f"Dias disponíveis: {profile.days_per_week} dias/semana")
print(f"Tempo disponível: {profile.hours_per_day}h/dia")
print(f"{'='*70}\n")

# Gerar plano COM UserProfile - o sistema irá automaticamente:
# 1. Calcular zonas de treino dos tempos informados
# 2. Ajustar volume baseado em lesões e IMC
# 3. Limitar duração de treinos baseado em tempo disponível
# 4. Recomendar dias de descanso adequados
# 5. Adicionar avisos sobre lesões
plan = PlanGenerator.generate_plan(
    name=f"Plano {profile.main_race.distance} - {profile.name}",
    goal=profile.main_race.distance,
    level=profile.experience_level,
    weeks=None,  # Será calculado automaticamente baseado na data da prova
    days_per_week=profile.days_per_week,
    training_zones=None,  # Será calculado do perfil automaticamente
    user_profile=profile  # NOVO: passa o perfil completo para personalização
)

# Definir data de início (próxima segunda-feira)
dias_ate_segunda = (7 - hoje.weekday()) % 7
if dias_ate_segunda == 0:
    dias_ate_segunda = 7
proxima_segunda = hoje + timedelta(days=dias_ate_segunda)
plan.set_start_date(datetime.combine(proxima_segunda, datetime.min.time()))

print("✓ Plano gerado com sucesso!\n")
print("🎯 Personalizações aplicadas:")
print(f"  • Duração do plano: {plan.weeks} semanas")
print(f"  • Dias de treino: {plan.days_per_week} dias/semana")
print(f"  • Início: {plan.start_date.strftime('%d/%m/%Y')}")
print(f"  • Prova: {plan.get_race_date().strftime('%d/%m/%Y')}")

# Mostrar ajustes do perfil
needs_mod, modifications = profile.needs_modified_plan()
if needs_mod:
    print("\n⚙️  Ajustes aplicados ao plano:")
    for mod in modifications:
        print(f"  • {mod}")

print("")

---
## 1️⃣1️⃣ Visualizar Plano

Veja seu plano de treino personalizado!

In [ ]:
# Mostrar visão geral do plano (formato visual compacto)
plan.print_visual()

### Ver Todas as Semanas em Formato Visual

In [ ]:
# Mostrar TODAS as semanas
plan.print_visual(show_all_weeks=True)

---
## 1️⃣2️⃣ Salvar Plano

Salve seu plano para uso futuro.

In [ ]:
# Salvar plano
plan_filename = f"plano_{profile.main_race.distance.lower().replace(' ', '_')}_{profile.name.lower().replace(' ', '_')}.json"
plan.save_to_file(plan_filename)

print(f"✓ Plano salvo em: {plan_filename}")
print(f"\n🎉 PARABÉNS! Seu plano personalizado está pronto!")
print(f"\n📋 Resumo:")
print(f"  • Meta: {profile.main_race.distance} em {profile.main_race.date.strftime('%d/%m/%Y')}")
print(f"  • Duração: {plan.weeks} semanas")
print(f"  • Dias de treino: {plan.days_per_week} dias/semana")
print(f"  • Início: {plan.start_date.strftime('%d/%m/%Y')}")
print(f"  • Prova: {plan.get_race_date().strftime('%d/%m/%Y')}")

total_km = sum(w.total_distance_km for w in plan.schedule)
print(f"  • Kilometragem total: {total_km:.1f}km")

print(f"\n💡 Dicas:")
print(f"  • Siga o plano consistentemente")
print(f"  • Escute seu corpo e descanse quando necessário")
print(f"  • Hidrate-se bem e alimente-se adequadamente")
print(f"  • Use o formato visual para acompanhar seu plano diariamente")

if profile.current_injuries:
    print(f"\n⚠️  IMPORTANTE: Você tem lesões atuais. Consulte um médico antes de iniciar.")

print(f"\n🏃‍♂️ Bons treinos e sucesso na sua prova!")